In [10]:
import ECGDataset 
import Models 
import Net
from train_test_validat import *
from self_attention import *
import matplotlib.pyplot as plt
import ecg_plot
import cam
import ECGplot
import ECGHandle
import torch
import torch.utils.data as Data
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold
import random
import pandas as pd
from tqdm import tqdm

import time
import math
import os
import gc
from torch.utils.tensorboard import SummaryWriter


def seed_torch(seed=2023):
	random.seed(seed)
	os.environ['PYTHONHASHSEED'] = str(seed) # 为了禁止hash随机化，使得实验可复现
	np.random.seed(seed)
	torch.manual_seed(seed)
	torch.cuda.manual_seed(seed)
	torch.cuda.manual_seed_all(seed) # if you are using multi-GPU.
	torch.backends.cudnn.benchmark = False 
	torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.enabled = False

In [11]:
EcgChannles_num = 12
EcgLength_num = 5000
DEVICE = "cpu"
seed_torch(2023)

In [12]:
data_root = '/workspace/data/Preprocess_HTN/datas_/'
ALL_data = pd.read_csv(data_root+'/All_data_handled_ID_range_age_IDimputate.csv',low_memory=False)
ALL_data = ECGHandle.change_label(ALL_data)
ALL_data = ECGHandle.filter_ID(ALL_data)
ALL_data = ECGHandle.filter_departmentORlabel(ALL_data,'外科')
ALL_data = ECGHandle.filter_ages(ALL_data,18) 




            orginal   removed diagnose NaN
   nums      200082          199997       
              HTN             NHTN        
   nums       3273           196724       


            orginal      removed ID NaN   
   nums      199997          199995       
              HTN             NHTN        
   nums       3273           196722       


            orginal   filtered department 
   nums      199995          45571        
              HTN             NHTN        
   nums       3273           42298        


            orginal      filtered ages    
   nums      45571           43570        
              HTN             NHTN        
   nums       3220           40350        


In [13]:
ALL_data = ECGHandle.filter_QC(ALL_data)
ALL_data = ECGHandle.correct_label(ALL_data)
ALL_data = ECGHandle.correct_age(ALL_data)
ALL_data = ECGHandle.filter_diagnose(ALL_data,'起搏')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'房颤')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左束支传导阻滞')
ALL_data = ECGHandle.filter_diagnose(ALL_data,'左前分支阻滞')



            orginal            QC         
   nums      43570           15344        
              HTN             NHTN        
   nums       1477           13867        


     reset num:       10  
  ERR labels num:     27  
            orginal      correct label    
   nums      15344           15344        
              HTN             NHTN        
   nums       1513           13831        


   ERR ages num:      831 
            orginal       correct age     
   nums      15344           15344        
              HTN             NHTN        
   nums       1513           13831        


            orginal    remove diagnose起搏  
   nums      15344           15278        
              HTN             NHTN        
   nums       1494           13784        
           remove HTN     remove NHTN     
   nums        19              47         


            orginal    remove diagnose房颤  
   nums      15278           14747        
              HTN             NHTN        
   num

In [14]:
ALL_data[ALL_data['label'] == 0]['住院号'].unique().__len__()

11254

In [15]:
ALL_data = ALL_data.rename(columns={'住院号':'ID','年龄':'age','性别':'gender','姓名':'name'}) 
ALL_data_buffer = ALL_data.copy()
seed_torch(2023)
ALL_data_buffer = ALL_data_buffer.sample(frac=1).reset_index(drop=True) #打乱顺序
# all_dataset = ECGHandle.ECG_Dataset(data_root,ALL_data_buffer,preprocess = True)
####################################################################随机选取test
test_df,tv_df = Pair_ID(ALL_data,0.2,Range_max=15,pair_num=1)
test_dataset = ECGHandle.ECG_Dataset(data_root,test_df,preprocess = True)

In [25]:
test_dataset.datas[:,0].std()

tensor(1.0000)

In [26]:
test_dataset.datas[:,0].mean()

tensor(1.2801e-09)

In [8]:
FOLDS = 5
seed_torch(2022)
tv_df = tv_df.sample(frac=1).reset_index(drop=True) #打乱顺序

In [9]:
for fold in range(FOLDS):
    print("Fold "+str(fold)+" of "+str(FOLDS) + ' :')
    tv_df_buffer = tv_df.copy()
    HTN_tv_df = tv_df[(tv_df['label']==1) ].copy()
    NHTN_tv_df = tv_df[(tv_df['label']==0) ].copy()
    HTN_ID_tv_list = HTN_tv_df['ID'].unique().tolist() #tvset中所有的HTN的ID号
    HTN_tv_size = HTN_tv_df['ID'].unique().__len__()
    HTN_validate_size = int(HTN_tv_size//FOLDS)
    validate_start_index = HTN_validate_size*fold #star index for validate
    validate_df,tarin_df = Pair_ID(tv_df_buffer,0.2,star_index=validate_start_index,Range_max=15,pair_num=1)
    validate_dataset = ECGHandle.ECG_Dataset(data_root,validate_df,preprocess = True)
    
    train_pair_df,_ = Pair_ID(tarin_df,1,star_index=0,Range_max=15,pair_num=1,shuffle=True)
    train_dataset = ECGHandle.ECG_Dataset(data_root,train_pair_df ,preprocess = True)
    for i in range(12):
        print('lead:' ,i)
        print('test:{}',test_dataset.datas[:,i,:].std(),test_dataset.datas[:,i,:].mean())
        print('train:{}',train_dataset.datas[:,i,:].std(),train_dataset.datas[:,i,:].mean())
        print('validat:{}',validate_dataset.datas[:,i,:].std(),validate_dataset.datas[:,i,:].mean())

Fold 0 of 5 :
lead: 0
test:{} tensor(1.0000) tensor(1.2801e-09)
train:{} tensor(1.) tensor(-3.1651e-09)
validat:{} tensor(1.0000) tensor(-1.8780e-09)
lead: 1
test:{} tensor(1.0000) tensor(-5.1206e-09)
train:{} tensor(1.0000) tensor(2.5320e-09)
validat:{} tensor(1.0000) tensor(1.9768e-09)
lead: 2
test:{} tensor(1.0000) tensor(-5.8521e-09)
train:{} tensor(1.) tensor(6.9056e-10)
validat:{} tensor(1.0000) tensor(-6.7213e-09)
lead: 3
test:{} tensor(1.0000) tensor(1.4630e-09)
train:{} tensor(1.) tensor(3.1075e-09)
validat:{} tensor(1.0000) tensor(1.9768e-09)
lead: 4
test:{} tensor(1.0000) tensor(7.3151e-10)
train:{} tensor(1.) tensor(-2.3019e-09)
validat:{} tensor(1.0000) tensor(1.7792e-09)
lead: 5
test:{} tensor(1.0000) tensor(0.)
train:{} tensor(1.) tensor(4.8339e-09)
validat:{} tensor(1.0000) tensor(9.8842e-10)
lead: 6
test:{} tensor(1.0000) tensor(6.4007e-10)
train:{} tensor(1.0000) tensor(4.1433e-09)
validat:{} tensor(1.0000) tensor(-2.6687e-09)
lead: 7
test:{} tensor(1.0000) tensor(-2.

In [34]:
test_df[test_df['label'] == 0]['ID'].__len__()

263

In [37]:
tv_df[tv_df['label'] == 0]['ID'].__len__()

12916